In [19]:
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from keras import callbacks
import keras_tuner as kt

In [3]:
dataset = pd.read_csv('grid.csv')

dataset['stabf'] = dataset['stabf'].replace({'unstable': 0, 'stable': 1})

In [4]:
from sklearn.model_selection import train_test_split

X = dataset.drop('stab', axis = 1)
y = X.pop('stabf')

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle =True, test_size=0.3, random_state=42)
X.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923


In [5]:
# mlp_model = Sequential(
#     [
#         Dense(24 ,input_dim = 12, activation='relu'),
#         Dense(24 , activation='relu'),
#         Dense(24 , activation='relu'),
#         Dense(12 , activation='relu'),
#         Dense(1 , activation='sigmoid')
#     ]
# )

# mlp_model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [6]:
early_stopping = callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=20, # n epochs before stopping
    restore_best_weights=True,
)

# mlp_model.fit(X_train,y_train,epochs = 150, callbacks=[early_stopping],validation_split=0.2)

In [7]:
from sklearn.metrics import classification_report

# y_pred = mlp_model.predict(X_test)
# y_pred = (y_pred > 0.5)

# report = classification_report(y_test,y_pred, output_dict = True)
# cr = pd.DataFrame(report).transpose()
# print(cr)

# Hyper Param Tuning

In [45]:
def model_builder(hp):
    hp_units = hp.Int('units', min_value=12, max_value=192, step=12)
    
    model = Sequential([
    Dense(units=hp_units ,input_dim = 12, activation='relu'),
    Dense(units=hp_units , activation='relu'),
    Dropout(0.1, seed=42),
    Dense(units=hp_units , activation='relu'),
    Dense(units=hp_units , activation='relu'),
    Dropout(0.1, seed=42),
    Dense(units=hp_units , activation='relu'),
    Dense(units=hp_units , activation='relu'),
    Dropout(0.1, seed=42),
    Dense(units=hp_units , activation='relu'),
    Dense(units=hp_units , activation='relu'),
    Dropout(0.1, seed=42),
    Dense(units=hp_units , activation='relu'),
    Dense(units=hp_units/2 , activation='relu'),
    Dense(1 , activation='sigmoid')
    ])

    
    

    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5])

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate),
                loss=BinaryCrossentropy(),
                metrics=['accuracy'])

    return model

In [46]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=300,)

tuner.search(X_train, y_train, epochs=150, validation_split=0.2, callbacks=[early_stopping])

Trial 40 Complete [00h 00m 05s]
val_accuracy: 0.8676190376281738

Best val_accuracy So Far: 0.9479761719703674
Total elapsed time: 00h 03m 08s
INFO:tensorflow:Oracle triggered exit


In [52]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

tuned_model = tuner.hypermodel.build(best_hps)
history = tuned_model.fit(X_train, y_train, epochs=150, callbacks=[early_stopping], validation_split=0.2)

Epoch 1/150
1050/1050 [==============================] - 3s 2ms/step - loss: 0.3323 - accuracy: 0.8497 - val_loss: 0.2775 - val_accuracy: 0.8793
Epoch 2/150
1050/1050 [==============================] - 2s 2ms/step - loss: 0.1929 - accuracy: 0.9180 - val_loss: 0.1429 - val_accuracy: 0.9365
Epoch 3/150
1050/1050 [==============================] - 2s 2ms/step - loss: 0.1566 - accuracy: 0.9335 - val_loss: 0.1573 - val_accuracy: 0.9271
Epoch 4/150
1050/1050 [==============================] - 2s 2ms/step - loss: 0.1362 - accuracy: 0.9435 - val_loss: 0.1090 - val_accuracy: 0.9558
Epoch 5/150
1050/1050 [==============================] - 2s 2ms/step - loss: 0.1253 - accuracy: 0.9470 - val_loss: 0.1085 - val_accuracy: 0.9458
Epoch 6/150
1050/1050 [==============================] - 2s 2ms/step - loss: 0.1250 - accuracy: 0.9473 - val_loss: 0.1838 - val_accuracy: 0.9357
Epoch 7/150
1050/1050 [==============================] - 2s 2ms/step - loss: 0.1161 - accuracy: 0.9520 - val_loss: 0.1092 - val_ac

1050/1050 [==============================] - 2s 2ms/step - loss: 0.0653 - accuracy: 0.9730 - val_loss: 0.0926 - val_accuracy: 0.9690
Epoch 58/150
1050/1050 [==============================] - 3s 2ms/step - loss: 0.0735 - accuracy: 0.9715 - val_loss: 0.0880 - val_accuracy: 0.9682
Epoch 59/150
1050/1050 [==============================] - 2s 2ms/step - loss: 0.0609 - accuracy: 0.9747 - val_loss: 0.0686 - val_accuracy: 0.9761
Epoch 60/150
1050/1050 [==============================] - 2s 2ms/step - loss: 0.0594 - accuracy: 0.9753 - val_loss: 0.0851 - val_accuracy: 0.9706
Epoch 61/150
1050/1050 [==============================] - 2s 2ms/step - loss: 0.0698 - accuracy: 0.9719 - val_loss: 0.0862 - val_accuracy: 0.9669
Epoch 62/150
1050/1050 [==============================] - 3s 2ms/step - loss: 0.0584 - accuracy: 0.9762 - val_loss: 0.1315 - val_accuracy: 0.9649
Epoch 63/150
1050/1050 [==============================] - 2s 2ms/step - loss: 0.0576 - accuracy: 0.9768 - val_loss: 0.0972 - val_accuracy

In [54]:
tuned_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_22 (Dense)            (None, 180)               2340      
                                                                 
 dense_23 (Dense)            (None, 180)               32580     
                                                                 
 dropout_8 (Dropout)         (None, 180)               0         
                                                                 
 dense_24 (Dense)            (None, 180)               32580     
                                                                 
 dense_25 (Dense)            (None, 180)               32580     
                                                                 
 dropout_9 (Dropout)         (None, 180)               0         
                                                                 
 dense_26 (Dense)            (None, 180)              

In [56]:
y_pred = tuned_model.predict(X_test)
y_pred = (y_pred > 0.5)

report = classification_report(y_test,y_pred, output_dict = True)
cr = pd.DataFrame(report).transpose()
print(cr)

              precision    recall  f1-score       support
0              0.976024  0.985973  0.980973  11478.000000
1              0.974863  0.957375  0.966040   6522.000000
accuracy       0.975611  0.975611  0.975611      0.975611
macro avg      0.975444  0.971674  0.973507  18000.000000
weighted avg   0.975604  0.975611  0.975563  18000.000000


# Droput Model

In [17]:
mlp_model_dropout = Sequential(
    [
        Dense(24 ,input_dim = 12, activation='relu'),
        Dense(24 , activation='relu'),
        Dropout(0.1, seed=42),
        Dense(24 , activation='relu'),
        Dense(24 , activation='relu'),
        Dropout(0.1, seed=42),
        Dense(24 , activation='relu'),
        Dense(24 , activation='relu'),
        Dropout(0.1, seed=42),
        Dense(24 , activation='relu'),
        Dense(24 , activation='relu'),
        Dropout(0.1, seed=42),
        Dense(24 , activation='relu'),
        Dense(12 , activation='relu'),
        Dense(1 , activation='sigmoid')
    ]
)

mlp_model_dropout.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

mlp_model_dropout.fit(X_train,y_train,epochs = 150, callbacks=[early_stopping],validation_split=0.2)

Epoch 1/150
1050/1050 [==============================] - 2s 1ms/step - loss: 0.4217 - accuracy: 0.7996 - val_loss: 0.2929 - val_accuracy: 0.8683
Epoch 2/150
1050/1050 [==============================] - 1s 1ms/step - loss: 0.2475 - accuracy: 0.8933 - val_loss: 0.1990 - val_accuracy: 0.9149
Epoch 3/150
1050/1050 [==============================] - 1s 1ms/step - loss: 0.2003 - accuracy: 0.9134 - val_loss: 0.2239 - val_accuracy: 0.8988
Epoch 4/150
1050/1050 [==============================] - 1s 1ms/step - loss: 0.1801 - accuracy: 0.9228 - val_loss: 0.1701 - val_accuracy: 0.9327
Epoch 5/150
1050/1050 [==============================] - 1s 1ms/step - loss: 0.1665 - accuracy: 0.9304 - val_loss: 0.1601 - val_accuracy: 0.9349
Epoch 6/150
1050/1050 [==============================] - 1s 1ms/step - loss: 0.1574 - accuracy: 0.9339 - val_loss: 0.1779 - val_accuracy: 0.9187
Epoch 7/150
1050/1050 [==============================] - 1s 1ms/step - loss: 0.1493 - accuracy: 0.9367 - val_loss: 0.1841 - val_ac

In [29]:
mlp_model_dropout.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_33 (Dense)            (None, 24)                312       
                                                                 
 dense_34 (Dense)            (None, 24)                600       
                                                                 
 dropout_12 (Dropout)        (None, 24)                0         
                                                                 
 dense_35 (Dense)            (None, 24)                600       
                                                                 
 dense_36 (Dense)            (None, 24)                600       
                                                                 
 dropout_13 (Dropout)        (None, 24)                0         
                                                                 
 dense_37 (Dense)            (None, 24)               

In [18]:
y_pred = mlp_model_dropout.predict(X_test)
y_pred = (y_pred > 0.5)

report = classification_report(y_test,y_pred, output_dict = True)
cr = pd.DataFrame(report).transpose()
print(cr)

              precision    recall  f1-score       support
0              0.977377  0.982401  0.979883  11478.000000
1              0.968745  0.959982  0.964343   6522.000000
accuracy       0.974278  0.974278  0.974278      0.974278
macro avg      0.973061  0.971191  0.972113  18000.000000
weighted avg   0.974249  0.974278  0.974252  18000.000000
